# Lab 10 - Curiosity-driven exploration

This lab will exlore how the win-stay, lose-switch (WSLS) approach to the exploration-exploitation dilemma boosts curious exploration in our little bacteria friends.

Sections:
1. Reinforcement as a search strategy.
2. Curiosity as a search strategy.
3. Comparing all the methods we have learned so far.




## Background
In this final lab we take on all the agents that we have studied so far and some new ones too.


_The decisions to be made this week are the exact opposite of every other lab_.

I am giving you six tuned agents, and three "levers" which control the environment. The now familiar scent grid. Your job this week is to see how curiosity and reward learning bias our search.

Our target metrics:
- It must gather the most total reward, by a clear margin (error baar overlap)
- It must not die the most. That is, as long as one other agent dies more often, or all agents die 0 times, we'll call that good enough. (Any experimental trial which does not lead to finding at least a single target (aka reward) means the exploring agent dies. It's a harsh noisy world we live in, after all.)

Once again, on final time it's time for _taxic explorations_. We revisit the sniff world (aka _ScentGrid_) with a familiar twist. We look again at what happens when sense information is not just noisy, but suddenly missing altogether. A concrete, cheap to simulate, case of this is turbulent flows.


### Our agents, this time
We will study six agents. They are,

- A diffusion walker (aka rando-taxis) (aka _DiffusionGrid_)
- Sniff! (aka chemo-taxis) (aka _GradientDiffusionGrid_)
- Air cognition! (aka "smart" chemo-taxis) (aka _AccumulatorGradientGrid_)
- Info cognition! (aka "smart" info-taxis) (aka _AccumulatorInfoGrid_)
- RL w/ random softmax search (aka _ActorCriticGrid_)
- Curiosity and RL union (aka rewardo- and info-taxis) (aka _WSLSGrid_)

The goal is, as I said, to _the change the world_ -- until each agent "wins" (defined above).

### Our agents, in review

**Random search** (rando-taxis): Actions are sampled from an exponential distribution. For the _randotaxis_ agent number of steps means the number of steps or actions the agent takes.

**Sniff!** (chemo-taxis): Recall our basic model of E. Coli exploration is as simple as can be.

- When the gradient is positive, meaning you are going "up" the gradient, the probability of turning is set to _p pos_.
- When the gradient is negative, the turning probability is set to _p neg_. (See code below, for an example).
- If the agent "decides" to turn, the direction it takes is uniform random.
- The length of travel before the next turn decision is sampled from an exponential distribution just like the _DiffusionGrid_

**Costly cognition** ("smart", chemo- and info-taxis): Both _chemo-_ and _infotaxis_ agents will use a DDM-style accumulator to try and make better decisions about the direction of the gradient. These decisions are of course statistical in nature. (We won't be tuning the accumulator parameters in this lab. Assume the parameters I give you, for the DDM, are "good enough".)

As beforee will assume that the steps are in a sense conserved. For the other two (accumulator) agents a step can mean two things. For accumulator agents a step can be spent sampling/weighing noisy scent evidence in the same location, or it can be spent moving to a new location. _Note_: Even though the info-accumulator is more complex, it can take advantage of missing scent information to drive its behavior. It can also use positive scent hits, of course, too.

**RL** (rewardo-taxis): A Q-learning agent with softmax exploration. The RL agent has no shaping function, or intrinsic reward. It does not use the scent, in other words. It learns to value each position on the grid and make its immediate choices based on the value of the four possible actions that it can make (up, down, left, right).

**WSLS** (reward- and info-taxis): A agent that alternates between info-taxis and Q-learning. Both are deterministic. Exploration and exploitation without any random search, in other words.

_Details_: For this model a memory $M$ is a discrete probability distribution. I define information value $E$ on the norm of the derivative ($\nabla M), approximated by $\hat E = || f(x, M) - M ||$, where $||.||$ denotes the norm. (Norms are distances like hypotanooses.)

The goal of any info-taxis (aka, curiosity agent) is to maximize $E$, I claim, based on a Bellman-optimal policy $\pi^*_E$.

So armed with $\hat E$ I write down another (meta) policy $\pi^{\pi}$, in terms of a mixed series of values, $\hat E$ and environmental rewards $R$. This WSLS rule is shown below. The reward (exploit) policy $\pi_R$ is Q learning, same as for **RL**.

$$
    \begin{split}
        \Pi_{\pi} =
        \begin{cases}
            \pi^*_{\hat{E}} & : \hat{E} - \eta > R + \rho \\
            \pi_R 	& : \hat{E} - \eta < R + \rho \\
        \end{cases}
    \end{split}
$$




## Install and import needed modules

In [ ]:
# Install explorationlib?
!pip install --upgrade git+https://github.com/parenthetical-e/explorationlib
!pip install --upgrade git+https://github.com/MattChanTK/gym-maze.git

In [ ]:
import shutil
import glob
import os

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from copy import deepcopy

import explorationlib
from explorationlib.local_gym import ScentGrid

from explorationlib.agent import WSLSGrid
from explorationlib.agent import CriticGrid
from explorationlib.agent import SoftmaxActor
from explorationlib.agent import DiffusionGrid
from explorationlib.agent import GradientDiffusionGrid
from explorationlib.agent import AccumulatorGradientGrid
from explorationlib.agent import AccumulatorInfoGrid
from explorationlib.agent import ActorCriticGrid

from explorationlib.run import experiment
from explorationlib.util import select_exp
from explorationlib.util import load
from explorationlib.util import save

from explorationlib.local_gym import uniform_targets
from explorationlib.local_gym import constant_values
from explorationlib.local_gym import ScentGrid
from explorationlib.local_gym import create_grid_scent
from explorationlib.local_gym import add_noise
from explorationlib.local_gym import create_grid_scent_patches

from explorationlib.plot import plot_position2d
from explorationlib.plot import plot_length_hist
from explorationlib.plot import plot_length
from explorationlib.plot import plot_targets2d
from explorationlib.plot import plot_scent_grid
from explorationlib.plot import plot_targets2d

from explorationlib.score import total_reward
from explorationlib.score import num_death

In [ ]:
# Pretty plots
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.greedy=True
plt.rcParams["axes.facecolor"] = "white"
plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["font.size"] = "16"

# Dev
%load_ext autoreload
%autoreload 2

## Section 1 -  RL as a search strategy

### RL

To build some intuition, let's plot the the behavoir of our RL agent as it learns where the rewards are in a (fixed) ScentGrid env. The noise level is 2 standard deviaions, all but 10 percent of it deleted.




#### Question 1.1
Does the fact that the noise level of the scents is 2 standard deviaions, and all but 10 percent of it deleted matter for the RL agent?

In [ ]:
# Write your answer here as a comment. Explain yourself.

### Shared params and env

Okay, let's get started by setting up our environment. his is the standard scent grid we have used in prior labs.

In [ ]:
# Noise and delete
p_scent = 0.1
noise_sigma = 2.0

# Shared
num_experiments = 100
num_steps = 200
seed_value = 5838
num_targets = 20 # with 80 agents are more competitive!

# ! (leave alone)
detection_radius = 1
cog_mult = 1
max_steps = 1
min_length = 1
target_boundary = (10, 10)

# Targets
prng = np.random.RandomState(seed_value)
targets = uniform_targets(num_targets, target_boundary, prng=prng)
values = constant_values(targets, 1)

# Scents
scents = []
for _ in range(len(targets)):
    coord, scent = create_grid_scent_patches(
        target_boundary, p=1.0, amplitude=1, sigma=2)
    scents.append(scent)

# Env
env = ScentGrid(mode=None)
env.seed(seed_value)
env.add_scents(targets, values, coord, scents, noise_sigma=noise_sigma)

### Getting to know you, RL

For this demo we will set up two agents:

- Rando: random walker
- RL: An agent that uses reinforcement learning to track a target. It updates the value of each point on the grid based on if it hit a target there before. This updates the immediate $Q$-value for each decision (up, down, left, right)

We are going to give each of our agents 99 tries at the _same_ environment. We want to see how repeated exposure to the same environment will improve performance in our agents.

In [ ]:
# RL
possible_actions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
critic = CriticGrid(default_value=0.5)
actor = SoftmaxActor(num_actions=4, actions=possible_actions, beta=4)
rl = ActorCriticGrid(actor, critic, lr=0.1, gamma=0.1)

# Rando
diff = DiffusionGrid(min_length=min_length, scale=1)
diff.seed(seed_value)

# !
rl_exp = experiment(
    f"RL",
    rl,
    env,
    num_steps=num_steps,
    num_experiments=num_experiments,
    dump=False,
    split_state=True,
    seed=seed_value
)
rand_exp = experiment(
    f"rand",
    diff,
    env,
    num_steps=num_steps,
    num_experiments=num_experiments,
    dump=False,
    split_state=True,
    seed=seed_value
)

### Rando search
Just one example of the movement of our random agent, for comparison with the cells below

In [ ]:
plot_boundary = (20, 20)

# -
num_experiment = 99
ax = None
ax = plot_position2d(
    select_exp(rand_exp, num_experiment),
    boundary=plot_boundary,
    label=f"Rando",
    color="grey",
    alpha=0.6,
    ax=ax,
)
ax = plot_targets2d(
    env,
    boundary=plot_boundary,
    color="black",
    alpha=1,
    label="Targets",
    ax=ax,
)

### Search behavior and learning


We will now look at the performance of our RL agent at three time points (N=0, early; N=50, middle; N=99, late)

In [ ]:
plot_boundary = (20, 20)

# -
num_experiment = 0
ax = None
ax = plot_position2d(
    select_exp(rl_exp, num_experiment),
    boundary=plot_boundary,
    label=f"N={num_experiment}",
    color="orange",
    alpha=0.3,
    ax=ax,
)
num_experiment = 50
ax = plot_position2d(
    select_exp(rl_exp, num_experiment),
    boundary=plot_boundary,
    label=f"N={num_experiment}",
    color="orange",
    alpha=0.5,
    ax=ax,
)
num_experiment = 99
ax = plot_position2d(
    select_exp(rl_exp, num_experiment),
    boundary=plot_boundary,
    label=f"N={num_experiment}",
    color="orange",
    alpha=1,
    ax=ax,
)
ax = plot_targets2d(
    env,
    boundary=plot_boundary,
    color="black",
    alpha=1,
    label="Targets",
    ax=ax,
)

### Reward value, in time

Now let's look at the value ($Q$-value in this case) of the optimal value (i.e., $max(Q(a))$) across time for each of these three stages of learning.


In [ ]:
fig = plt.figure(figsize=(6, 3))
plt.plot(rl_exp[0]["agent_reward_value"], label="N=0", color="orange", alpha=0.2)
plt.plot(rl_exp[50]["agent_reward_value"], label="N=50", color="orange", alpha=0.5)
plt.plot(rl_exp[99]["agent_reward_value"], label="N=99", color="orange", alpha=1)
plt.ylabel("Value $V(x)$")
plt.xlabel("Step")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))


#### Question 1.2

What do you see in this behavior of the RL agent over time?

In [ ]:
# Write your answer here as a comment. Explain yourself.

### Looking across simulations

Now let's plot our metrics and see how the two agents did.

### Death

In [ ]:
# Results
results = [rand_exp, rl_exp]
names = ["Rando", "RL"]
colors = ["grey", "orange"]

# Score by eff
scores = []
for name, res, color in zip(names, results, colors):
    scores.append(num_death(res))

# Tabulate
m, sd = [], []
for (name, s, c) in zip(names, scores, colors):
    m.append(np.mean(s))
    sd.append(np.std(s))

# Plot means
fig = plt.figure(figsize=(4, 3))
plt.bar(names, m, yerr=sd, color="black", alpha=0.6)
plt.ylabel("Deaths")
plt.tight_layout()
sns.despine()

### Total reward

In [ ]:
# Results
results = [rand_exp, rl_exp]
names = ["Rando", "RL"]
colors = ["grey", "orange"]

# Score by eff
scores = []
for name, res, color in zip(names, results, colors):
    r = total_reward(res)
    scores.append(r)

# Tabulate
m, sd = [], []
for (name, s, c) in zip(names, scores, colors):
    m.append(np.mean(s))
    sd.append(np.std(s))

# Plot means
fig = plt.figure(figsize=(3, 3))
plt.bar(names, m, yerr=sd, color=colors, alpha=0.6)
plt.ylabel("Total reward")
plt.tight_layout()
sns.despine()

# Dists
fig = plt.figure(figsize=(6, 3))
for (name, s, c) in zip(names, scores, colors):
    plt.hist(s, label=name, color=c, alpha=0.5, bins=np.linspace(0, np.max(scores), 50))
    plt.legend()
    plt.xlabel("Score")
    plt.tight_layout()
    sns.despine()


#### Question 1.3

How does the performance of the RL agent compare, across all performance metrics, to the random agent? Is this a fair comparison? Why or why not?

In [ ]:
# Write your answer here as a comment. Explain yourself.

## Section 2 -  WSLS as a search strategy

### WSLS

Now let's include our curious agent. Remember, this agent tracks the value of information as well as the value of rewards. It is basically two agents acting as one: the RL agent and an info-taxis agent.

The WSLS agnet makes a choice to go after whatever action has the highest value ($R$ or $E$), swtiching back and forth between seeking rewards (i.e., targets) and information.

In [ ]:
# WSLS
possible_actions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
num_action = len(possible_actions)
initial_bins = np.linspace(0, 1, 10)

critic_R = CriticGrid(default_value=0.0)
critic_E = CriticGrid(default_value=np.log(num_action))
actor_R = SoftmaxActor(num_actions=4, actions=possible_actions, beta=20)
actor_E = SoftmaxActor(num_actions=4, actions=possible_actions, beta=20)

wsls = WSLSGrid(
    actor_E,
    critic_E,
    actor_R,
    critic_R,
    initial_bins,
    lr=0.1,
    gamma=0.1,
    boredom=0.0
)

# Rando
diff = DiffusionGrid(min_length=min_length, scale=1)
diff.seed(seed_value)

# !
wsls_exp = experiment(
    f"wsls",
    wsls,
    env,
    num_steps=num_steps,
    num_experiments=num_experiments,
    dump=False,
    split_state=True,
    seed=seed_value
)
rand_exp = experiment(
    f"rand",
    diff,
    env,
    num_steps=num_steps,
    num_experiments=num_experiments,
    dump=False,
    split_state=True,
    seed=seed_value
)

Just like before, let's look at our random agent's behavior for later comparisons.

In [ ]:
plot_boundary = (20, 20)

# -
num_experiment = 99
ax = None
ax = plot_position2d(
    select_exp(rand_exp, num_experiment),
    boundary=plot_boundary,
    label=f"Rando",
    color="grey",
    alpha=0.6,
    ax=ax,
)
ax = plot_targets2d(
    env,
    boundary=plot_boundary,
    color="black",
    alpha=1,
    label="Targets",
    ax=ax,
)

### Reward AND Information value across learning

Just like before, let's see how the behavior of our WSLS agent changes across repeated exposures to the same environment.

In [ ]:
plot_boundary = (20, 20)

# -
num_experiment = 0
ax = None
ax = plot_position2d(
    select_exp(wsls_exp, num_experiment),
    boundary=plot_boundary,
    label=f"N={num_experiment}",
    color="orangered",
    alpha=0.3,
    ax=ax,
)
num_experiment = 50
ax = plot_position2d(
    select_exp(wsls_exp, num_experiment),
    boundary=plot_boundary,
    label=f"N={num_experiment}",
    color="orangered",
    alpha=0.5,
    ax=ax,
)
num_experiment = 99
ax = plot_position2d(
    select_exp(wsls_exp, num_experiment),
    boundary=plot_boundary,
    label=f"N={num_experiment}",
    color="orangered",
    alpha=1,
    ax=ax,
)
ax = plot_targets2d(
    env,
    boundary=plot_boundary,
    color="black",
    alpha=1,
    label="Targets",
    ax=ax,
)


#### Question 2.1

Based off of the behavior of our WSLS agent, how well do you think it will perform after repeated runs (compared to the Rando agent)?

In [ ]:
# Write your answer here as a comment. Explain yourself.

### Value over time

Again let's look at the optional Value driving the action ($R$ or $E$) of our agent.


In [ ]:
fig = plt.figure(figsize=(6, 3))
plt.plot(wsls_exp[0]["agent_reward_value"], label="N=0", color="orangered", alpha=0.2)
plt.plot(wsls_exp[50]["agent_reward_value"], label="N=50", color="orangered", alpha=0.5)
plt.plot(wsls_exp[99]["agent_reward_value"], label="N=99", color="orangered", alpha=1)
plt.ylabel("Value $V(x)$")
plt.xlabel("Step")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

### Death

In [ ]:
# Results
results = [rand_exp, wsls_exp]
names = ["Rando", "WSLS"]
colors = ["grey", "orangered"]

# Score by eff
scores = []
for name, res, color in zip(names, results, colors):
    scores.append(num_death(res))

# Tabulate
m, sd = [], []
for (name, s, c) in zip(names, scores, colors):
    m.append(np.mean(s))
    sd.append(np.std(s))

# Plot means
fig = plt.figure(figsize=(3, 3))
plt.bar(names, m, yerr=sd, color=colors, alpha=0.9)
plt.ylabel("Deaths")
plt.tight_layout()
sns.despine()

### Total reward

In [ ]:
# Results
results = [rand_exp, wsls_exp]
names = ["Rando", "WSLS"]
colors = ["grey", "orangered"]

# Score by eff
scores = []
for name, res, color in zip(names, results, colors):
    r = total_reward(res)
    scores.append(r)

# Tabulate
m, sd = [], []
for (name, s, c) in zip(names, scores, colors):
    m.append(np.mean(s))
    sd.append(np.std(s))

# Plot means
fig = plt.figure(figsize=(3, 3))
plt.bar(names, m, yerr=sd, color=colors, alpha=0.6)
plt.ylabel("Total reward")
plt.tight_layout()
sns.despine()

# Dists
fig = plt.figure(figsize=(6, 4))
for (name, s, c) in zip(names, scores, colors):
    plt.hist(s, label=name, color=c, alpha=0.5, bins=np.linspace(0, np.max(scores), 50))
    plt.legend()
    plt.xlabel("Score")
    plt.tight_layout()
    sns.despine()

#### Question 2.2

Based on the performance you have seen above, is it better to be curious and greedy, or greedy and noisy?
Compare the RL agent to the WSLS agent (and Rando for control).

In [ ]:
# Write your answer here as a comment. Explain yourself.

## Now let's compare the agents directly.

### Death

In [ ]:
# Results
results = [rand_exp, rl_exp, wsls_exp]
names = ["Rando", "RL", "WSLS"]
colors = ["grey", "orange", "orangered"]

# Score by eff
scores = []
for name, res, color in zip(names, results, colors):
    scores.append(num_death(res))

# Tabulate
m, sd = [], []
for (name, s, c) in zip(names, scores, colors):
    m.append(np.mean(s))
    sd.append(np.std(s))

# Plot means
fig = plt.figure(figsize=(4, 3))
plt.bar(names, m, yerr=sd, color=colors, alpha=0.9)
plt.ylabel("Deaths")
plt.tight_layout()
sns.despine()

### Total reward

In [ ]:
# Results
results = [rand_exp, rl_exp, wsls_exp]
names = ["Rando", "RL", "WSLS"]
colors = ["grey", "orange", "orangered"]

# Score by eff
scores = []
for name, res, color in zip(names, results, colors):
    r = total_reward(res)
    scores.append(r)

# Tabulate
m, sd = [], []
for (name, s, c) in zip(names, scores, colors):
    m.append(np.mean(s))
    sd.append(np.std(s))

# Plot means
fig = plt.figure(figsize=(3, 3))
plt.bar(names, m, yerr=sd, color=colors, alpha=0.6)
plt.ylabel("Total reward")
plt.tight_layout()
sns.despine()

# Dists
fig = plt.figure(figsize=(6, 4))
for (name, s, c) in zip(names, scores, colors):
    plt.hist(s, label=name, color=c, alpha=0.5, bins=np.linspace(0, np.max(scores), 50))
    plt.legend()
    plt.xlabel("Score")
    plt.tight_layout()
    sns.despine()

#### Question 2.3
The WSLS approach should have generated more total reward. It may also have had a few (< 10) deaths. (If it did not, try running the WSLS cells again).

Likewise, if you study WSLS search behavior and value learning time courses, you'll see it "settles down" to one rewarding spot and can stay there.

In other words, WSLS is a method with very high inductive bias.

Based on the results in this lab so far, and lecture on managing the exploration-exploitation dilemma, how could you change the env so that the exploration bias behind WSLS (deterministic learning maximization) fails, but the random search of RL does not.

_Note:_ It is helpful to consider the total reward distribution plots carefully. The middle and the bottom range, especially. (Try rerunning?)

_Note_: Everything is on the table. Your counter-example can be whatever you want, well as long as it is physically possible. Be imaginative!

In [ ]:
# Write your answer here as a comment. Explain yourself.

## Section 3 - The battle royal of our bacteria agents

Now let's run all of our agents on the same world from Sections 1 and 2.

### Keep our environment the same

Just resetting here in case we tweaked parameters above.

In [ ]:
# Noise and delete
p_scent = 0.1
noise_sigma = 2.0

# Shared
num_experiments = 100
num_steps = 200
seed_value = 5838
num_targets = 20 # with 80 agents are more competitive!

# ! (leave alone)
detection_radius = 1
cog_mult = 1
max_steps = 1
min_length = 1
target_boundary = (10, 10)

# Targets
prng = np.random.RandomState(seed_value)
targets = uniform_targets(num_targets, target_boundary, prng=prng)
values = constant_values(targets, 1)

# Scents
scents = []
for _ in range(len(targets)):
    coord, scent = create_grid_scent_patches(
        target_boundary, p=1.0, amplitude=1, sigma=2)
    scents.append(scent)

# Env
env = ScentGrid(mode=None)
env.seed(seed_value)
env.add_scents(targets, values, coord, scents, noise_sigma=noise_sigma)

### Run 'em all!

Now we can run all 6 of our agent types in the environment, the same way, and see how well they did.

In [ ]:
# Agents

# rando
diff = DiffusionGrid(min_length=min_length, scale=1)
diff.seed(seed_value)

# sniff
sniff = GradientDiffusionGrid(
    min_length=min_length,
    scale=1.0,
    p_neg=1,
    p_pos=0.0
)
sniff.seed(seed_value)

# smart chemo
chemo = AccumulatorGradientGrid(
    min_length=min_length,
    max_steps=max_steps,
    drift_rate=1,
    threshold=3,
    accumulate_sigma=1
)
chemo.seed(seed_value)

# smart info
info = AccumulatorInfoGrid(
    min_length=min_length,
    max_steps=max_steps,
    drift_rate=1,
    threshold=3,
    accumulate_sigma=1
)
info.seed(seed_value)

# RL
critic = CriticGrid(default_value=0.5)
actor = SoftmaxActor(num_actions=4, actions=possible_actions, beta=4)
rl = ActorCriticGrid(actor, critic, lr=0.1, gamma=0.1)

# WSLS
possible_actions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
num_action = len(possible_actions)
initial_bins = np.linspace(0, 1, 10)

critic_R = CriticGrid(default_value=0.5)
critic_E = CriticGrid(default_value=np.log(num_action))
actor_R = SoftmaxActor(num_actions=4, actions=possible_actions, beta=20)
actor_E = SoftmaxActor(num_actions=4, actions=possible_actions, beta=20)

wsls = WSLSGrid(
    actor_E,
    critic_E,
    actor_R,
    critic_R,
    initial_bins,
    lr=0.1,
    gamma=0.1,
    boredom=0.0
)

# !
rand_exp = experiment(
    f"rand",
    diff,
    env,
    num_steps=num_steps,
    num_experiments=num_experiments,
    dump=False,
    split_state=True,
    seed=seed_value
)
sniff_exp = experiment(
    f"sniff",
    sniff,
    env,
    num_steps=num_steps,
    num_experiments=num_experiments,
    dump=False,
    split_state=True,
    seed=seed_value
)
chemo_exp = experiment(
    f"chemo",
    chemo,
    env,
    num_steps=num_steps * cog_mult,
    num_experiments=num_experiments,
    dump=False,
    split_state=True,
    seed=seed_value
)
info_exp = experiment(
    f"info",
    info,
    env,
    num_steps=num_steps * cog_mult,
    num_experiments=num_experiments,
    dump=False,
    split_state=True,
    seed=seed_value
)
rl_exp = experiment(
    f"rl",
    rl,
    env,
    num_steps=num_steps,
    num_experiments=num_experiments,
    dump=False,
    split_state=True,
    seed=seed_value
)
wsls_exp = experiment(
    f"wsls",
    wsls,
    env,
    num_steps=num_steps,
    num_experiments=num_experiments,
    dump=False,
    split_state=True,
    seed=seed_value
)

### Search behavior

Let's just take a look at the movements of each agent at the last run of the repeated tests.

In [ ]:
plot_boundary = (20, 20)
num_experiment = 99

# Results
results = [sniff_exp, chemo_exp, info_exp, rand_exp, rl_exp, wsls_exp]
names = ["Sniff", "Chemo", "Info", "Rando", "RL", "WSLS"]
colors = ["purple", "blue", "green", "grey", "orange", "orangered"]

for name, res, color in zip(names, results, colors):
    ax = None
    ax = plot_position2d(
        select_exp(res, num_experiment),
        boundary=plot_boundary,
        label=f"{name}",
        color=color,
        alpha=0.6,
        ax=ax,
    )
    ax = plot_targets2d(
        env,
        boundary=plot_boundary,
        color="black",
        alpha=1,
        label="Targets",
        ax=ax,
    )

#### Question 3.1

What do you see in the different agents' behavior? Who moved more and who moved less?

In [ ]:
# Write your answer here as a comment. Explain yourself.

## Now let's run one final evaluation to test them all

### Death

In [ ]:
# Results
results = [rand_exp, sniff_exp, chemo_exp, info_exp,  rl_exp, wsls_exp]
names = ["Rando", "Sniff", "Chemo", "Info",  "RL", "WSLS"]
colors = ["grey", "purple", "blue", "green", "orange", "orangered"]

# Score by eff
scores = []
for name, res, color in zip(names, results, colors):
    scores.append(num_death(res))

# Tabulate
m, sd = [], []
for (name, s, c) in zip(names, scores, colors):
    m.append(np.mean(s))
    sd.append(np.std(s))

# Plot means
fig = plt.figure(figsize=(6, 3))
plt.bar(names, m, yerr=sd, color=colors, alpha=0.6)
plt.ylabel("Deaths")
plt.tight_layout()
sns.despine()

### Total reward

In [ ]:
# Results
results = [rand_exp, sniff_exp, chemo_exp, info_exp,  rl_exp, wsls_exp]
names = ["Rando", "Sniff", "Chemo", "Info","RL", "WSLS"]
colors = ["grey","purple", "blue", "green", "orange", "orangered"]

# Score
scores = []
for name, res, color in zip(names, results, colors):
    r = total_reward(res)
    scores.append(r)

# Tabulate
m, sd = [], []
for (name, s, c) in zip(names, scores, colors):
    m.append(np.mean(s))
    sd.append(np.std(s))

# Plot means
fig = plt.figure(figsize=(6, 3))
plt.bar(names, m, yerr=sd, color=colors, alpha=0.6)
plt.ylabel("Total reward")
plt.tight_layout()
sns.despine()

# Dists
# fig = plt.figure(figsize=(7, 5))
for (name, s, c) in zip(names, scores, colors):
    fig = plt.figure(figsize=(7, 3))
    plt.hist(s, label=name, color=c, alpha=0.4, bins=np.linspace(0, np.max(scores), 50))
    plt.legend()
    plt.xlabel("Score")
    plt.tight_layout()
    sns.despine()

#### Question 3.2


In the same environment, we scaled up th ecomplexity of the decisions of our little bacteria friends. What pattern did you see across agents? Who did the best, who did worse, and who was most variable?

In [ ]:
# Write your answer here as a comment. Explain yourself.

#### Question 3.3

Remember from the prior labs that sometimes the Rando agent outperformed agents with more directed decisions in their search. What changes in the environment would provide an edge for the simpler (and more random) agents?

In [ ]:
# Write your answer here as a comment. Explain yourself.